In [30]:
import torch
from torchvision.io import read_image
from torchvision.utils import draw_segmentation_masks,draw_bounding_boxes
from torchvision.ops import masks_to_boxes
import torchvision.transforms.functional as F

import numpy as np
from path import Path
import matplotlib.pyplot as plt
plt.rcParams["savefig.bbox"] = "tight"

## 实例分割转目标检测

In [31]:
def show(imgs):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fix, axs = plt.subplots(ncols=len(imgs), squeeze=False)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = F.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

In [ ]:
maskpath=Path(r'0.png')
imagepath=Path(r'0.jpg')
mask=read_image(maskpath)
img=read_image(imagepath)

obj_ids = torch.unique(mask)

# first id is the background, so remove it.
obj_ids = obj_ids[1:]

# split the color-encoded mask into a set of boolean masks.
# Note that this snippet would work as well if the masks were float values instead of ints.
masks = mask == obj_ids[:, None, None]
masks.size()

In [ ]:

drawn_masks = []
for mask in masks:
    drawn_masks.append(draw_segmentation_masks(img, mask, alpha=0.8, colors="blue"))

show(drawn_masks)

In [ ]:
boxes = masks_to_boxes(masks)
print(boxes.size())
print(boxes)

In [ ]:
drawn_boxes = draw_bounding_boxes(img, boxes, colors="red")
show(drawn_boxes)

## 语义分割转目标检测

In [35]:
import cv2
from path import Path
import numpy as np
from skimage import morphology,measure
import matplotlib.pyplot as plt

In [36]:


def getboundingbox(imdata:np.array,min_size=10):
    mask=imdata>0
    # 删掉小于min_size的目标
    mask_without_small = morphology.remove_small_objects(mask,min_size=min_size,connectivity=2)
    # 连通域标记
    label_image = measure.label(mask_without_small,connectivity=2)
    #得到锚框
    regionprops= measure.regionprops(label_image)
    boundingbox = [region.bbox for region in regionprops]
    boundingbox = [[box[1],box[0],box[3],box[2]] for box in boundingbox]
    return boundingbox

In [37]:
def newgetboundingbox(imdata:np.array, min_size=2, ksize=(5,5)):
    # 通过膨胀去连通，可以min_size筛除小目标,或者通过锚框面积筛除
    # 通过kernel调整膨胀
    ## 膨胀
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, ksize=ksize)
    dilated = cv2.dilate(imdata.copy(), kernel, 1)
    # plt.imshow(dilated)
    bboxs = getboundingbox(dilated,min_size)
    # print(bboxs)
    ## 细化锚框
    tbboxs =getboundingbox(imdata,min_size)
    # print(tbboxs)
    cluster=[]
    for bbox in bboxs:
        tmp=[]
        for tbbox in tbboxs:
            if tbbox[0]>=bbox[0] and tbbox[1]>=bbox[1] and tbbox[2]<=bbox[2] and tbbox[3]<=bbox[3]:
                tmp.append(tbbox)
        cluster.append(tmp)

    res=[]
    for i in cluster:
        # print(i)
        xy=list(zip(*i))
        if xy:
            res.append([min(xy[0]),min(xy[1]),max(xy[2]),max(xy[3])])
    return res

In [ ]:
maskpath=Path(r'D:\nn\code\datasets\fire\satellite\seg\25.png')
im=cv2.imread(maskpath,0)

res=newgetboundingbox(im)

# 考虑DBSCAN是否可以实现

for box in res:
    cv2.rectangle(img=im,
                  pt1=(box[0], box[1]),
                  pt2=(box[2], box[3]),
                  color=(255, 0, 0), 
                  thickness=2)
    cv2.putText(img=im, 
                text='fire', 
                org=(box[0],box[1]-5),
                fontFace=cv2.FONT_HERSHEY_COMPLEX,
                fontScale=0.7,
                color=(255,0,0),
                thickness=1,
                lineType=cv2.LINE_AA)  
plt.imshow(im)

## 生成标注文件

In [47]:
import xml.etree.ElementTree as ET

def createXMLlabel(savedir,objectnum, bbox, classname=None, foldername='0',filename='0', path='0', database='road', width='400', height='600',depth='3', segmented='0', pose="Unspecified", truncated='0', difficult='0'):
    # 创建根节点
    root = ET.Element("annotation")
 
    # 创建子节点
    folder_node = ET.Element("folder")
    folder_node.text = foldername
    # 将子节点数据添加到根节点
    root.append(folder_node)
 
    file_node = ET.Element("filename")
    file_node.text = filename
    root.append(file_node)
    path_node = ET.Element("path")
    path_node.text = path
    root.append(path_node)
 
    source_node = ET.Element("source")
    # 也可以使用SubElement直接添加子节点
    db_node = ET.SubElement(source_node, "database")
    db_node.text = database
    root.append(source_node)
 
    size_node = ET.Element("size")
    width_node = ET.SubElement(size_node, "width")
    height_node = ET.SubElement(size_node, "height")
    depth_node = ET.SubElement(size_node, "depth")
    width_node.text = str(width)
    height_node.text = str(height)
    depth_node.text = str(depth)
    root.append(size_node)
 
    seg_node = ET.Element("segmented")
    seg_node.text = segmented
    root.append(seg_node)
    
    for i in range(objectnum):
        newEle = ET.Element("object")
        name = ET.Element("name")
        if classname:
            name.text = classname
        else:
            name.text=bbox[i][4]
        newEle.append(name)
        pose_node = ET.Element("pose")
        pose_node.text = pose
        newEle.append(pose_node)
        trunc = ET.Element("truncated")
        trunc.text = truncated
        newEle.append(trunc)
        dif = ET.Element("difficult")
        dif.text = difficult
        newEle.append(dif)
        boundingbox = ET.Element("bndbox")
        xmin = ET.SubElement(boundingbox, "xmin")
        ymin = ET.SubElement(boundingbox, "ymin")
        xmax = ET.SubElement(boundingbox, "xmax")
        ymax = ET.SubElement(boundingbox, "ymax")
        xmin.text = str(int(bbox[i][0]))
        ymin.text = str(int(bbox[i][1]))
        xmax.text = str(int(bbox[i][2]))
        ymax.text = str(int(bbox[i][3]))
        newEle.append(boundingbox)
        root.append(newEle)
 
    ImageID = filename.split('.')[0]
    # 创建elementtree对象，写入文件
    tree = ET.ElementTree(root)
    tree.write(savedir + '/'+ ImageID + ".xml")
 

In [48]:
root=Path(r'')
images = root/'JPEGImages'
masks = root/'Segmentations'
annos = root/'Annotations'
if not annos.exists():
    annos.mkdir()
for m in masks.files():

    mask = cv2.imread(m,0)
    bbox = newgetboundingbox(mask)

    objectnum = len(bbox)
    image_name = f'{m.stem}.jpg'
    height,width = mask.shape

    foldername = 'JPEGImages'
    impath  = images/image_name
    
    database = 'Unknown'
    depth = '3'
    classname='fire'
    
    createXMLlabel(annos,objectnum, bbox, classname, foldername=foldername,filename=image_name, path=impath,
                    database=database, width=width, height=height,depth=depth, segmented='0', pose="Unspecified",
                    truncated='0', difficult='0')